## 数据处理

In [1]:
import pandas as pd
import numpy as np

由于这个数据是二分类数据，最后一列是分类， g和b表示，字符串，不是float，所以无法使用np.loadtext来读取

In [2]:
data=pd.read_csv('./datasets/ionosphere.data',delimiter=',',header=None,encoding='utf8')

In [3]:
dataset=data.values

In [4]:
# 划分特征和标签
X=dataset[:,0:34].astype(float)
Y=dataset[:,34]

In [5]:
Y[:4]

array(['g', 'b', 'g', 'b'], dtype=object)

## 按照时间减少lr

In [6]:
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder

In [7]:
encoder=LabelEncoder()
Y=encoder.fit_transform(Y) #数字化

In [9]:
Y[:2]

array([1, 0], dtype=int64)

    lr: float >= 0. Learning rate.
    momentum: float >= 0. Parameter that accelerates SGD
        in the relevant direction and dampens oscillations.
    decay: float >= 0. Learning rate decay over each update.

In [13]:
model=Sequential()
model.add(Dense(34,input_dim=34,activation='relu',kernel_initializer='normal'))
model.add(Dense(1,activation='sigmoid',kernel_initializer='normal'))
sgd=SGD(lr=0.1,momentum=0.8,decay=0.002,nesterov=False) 
# 想训练50次，0.1/50=0.002 每次更新 学习率衰减0.002 50次刚好衰减完
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
model.fit(X,Y,epochs=50,validation_split=0.33,batch_size=28)

Train on 235 samples, validate on 116 samples
Epoch 1/50
235/235 [==============================] - 0s 1ms/step - loss: 0.6856 - acc: 0.5532 - val_loss: 0.6082 - val_acc: 0.9052
Epoch 2/50
235/235 [==============================] - 0s 68us/step - loss: 0.6334 - acc: 0.7319 - val_loss: 0.5437 - val_acc: 0.6810
Epoch 3/50
235/235 [==============================] - 0s 85us/step - loss: 0.5584 - acc: 0.7745 - val_loss: 0.4386 - val_acc: 0.8707
Epoch 4/50
235/235 [==============================] - 0s 77us/step - loss: 0.5108 - acc: 0.8000 - val_loss: 0.6731 - val_acc: 0.6293
Epoch 5/50
235/235 [==============================] - 0s 81us/step - loss: 0.4311 - acc: 0.8298 - val_loss: 0.3148 - val_acc: 0.9483
Epoch 6/50
235/235 [==============================] - 0s 81us/step - loss: 0.3636 - acc: 0.8766 - val_loss: 0.1934 - val_acc: 0.9569
Epoch 7/50
235/235 [==============================] - 0s 72us/step - loss: 0.2952 - acc: 0.8936 - val_loss: 0.4853 - val_acc: 0.7500
Epoch 8/50
235/235 [====

这个准确率很高了

## 基于轮数的学习速度调整

还是刚才的网络

In [15]:
from keras.callbacks import LearningRateScheduler
import math

In [16]:
def step_decay(epoch):
    initial_lr=0.1
    drop=0.5
    epochs_drop=10
    lr=initial_lr*math.pow(drop,math.floor((1+epoch)/epochs_drop))
    # 初始lr，drop是减速频率，epochs_drop是 降低多少
    return lr

    LearningRateScheduler(scheduler)参数说明：

    schedule: a function that takes an epoch index as input
            (integer, indexed from 0) and current learning rate
            and returns a new learning rate as output (float).

In [20]:
model=Sequential()
model.add(Dense(34,input_dim=34,activation='relu',kernel_initializer='normal'))
model.add(Dense(1,activation='sigmoid',kernel_initializer='normal'))
sgd=SGD(lr=0.0,momentum=0.9,decay=0.0,nesterov=False) 
# 想训练50次，0.1/50=0.002 每次更新 学习率衰减0.002 50次刚好衰减完
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])

lrate=LearningRateScheduler(step_decay)
model.fit(X,Y,epochs=50,validation_split=0.33,batch_size=28,callbacks=[lrate])

Train on 235 samples, validate on 116 samples
Epoch 1/50
235/235 [==============================] - 0s 2ms/step - loss: 0.6794 - acc: 0.6000 - val_loss: 0.5940 - val_acc: 0.9310
Epoch 2/50
235/235 [==============================] - 0s 81us/step - loss: 0.6189 - acc: 0.6936 - val_loss: 0.4563 - val_acc: 0.9397
Epoch 3/50
235/235 [==============================] - 0s 102us/step - loss: 0.4951 - acc: 0.8170 - val_loss: 0.4831 - val_acc: 0.8534
Epoch 4/50
235/235 [==============================] - 0s 81us/step - loss: 0.3719 - acc: 0.8681 - val_loss: 0.2948 - val_acc: 0.9138
Epoch 5/50
235/235 [==============================] - 0s 94us/step - loss: 0.2663 - acc: 0.9021 - val_loss: 0.2012 - val_acc: 0.9310
Epoch 6/50
235/235 [==============================] - 0s 102us/step - loss: 0.2183 - acc: 0.9021 - val_loss: 0.1219 - val_acc: 0.9655
Epoch 7/50
235/235 [==============================] - 0s 89us/step - loss: 0.1949 - acc: 0.9149 - val_loss: 0.1061 - val_acc: 0.9741
Epoch 8/50
235/235 [==

文中说的不对，这个模型sgd参数改了，而且sgd并不是没有起作用，如果将momentum参数变为0，则效果会变差